# Summarize YouTube video with GPT-3

In [ ]:
#@title Quick start { run: "auto" }
#@markdown <small>Fill out this form and then click Runtime -> Run all</small>

video_url = 'https://www.youtube.com/watch?v=_q1K7cybyRk' #@param {type:"string"}

openai_api_key = "sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX" #@param {type:"string"}

#@markdown <small>(Get an API key from http://beta.openai.com/account/api-keys)</small>

whisper_model = "large-v2" #@param ["large-v2", "medium.en", "small.en"]


# Setup imports

In [ ]:
#@title 1. Install libraries { display-mode: "form" }

!pip install youtube-dl
!pip install git+https://github.com/openai/whisper.git
!pip install openai
!pip install transformers

In [ ]:
# @title 2. Initial setup

import os
import numpy as np

try:
    import tensorflow
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import openai
import torchaudio

from tqdm.notebook import tqdm
from IPython.display import Markdown

import youtube_dl
from transformers import GPT2TokenizerFast

openai.api_key=openai_api_key

# Initialize Whisper Model
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model(whisper_model)

# Disable timestamps
options = whisper.DecodingOptions(language="en", without_timestamps=True)

# Tokenizer (for counting tokens)
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# Download audio and transcribe

In [ ]:
# @title Download audio from YouTube video

mp3_file_path = "./" + video_url.split("v=")[1].split("&")[0] + ".mp3"

ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': mp3_file_path,
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

In [ ]:
# @title Transcribe Audio

# Start transcription

result = model.transcribe(mp3_file_path)

transcript = result['text'].strip()

# Generate summary



In [ ]:
# @title Summary

prompt = f'''
The text below is transcribed from audio.

Instructions:

- Write a title that faithfully sums up the speaker's talk in the format "# <title>"

- Break the talk into sections with subheadings — make sure to format them as markdown!

- Rewrite each section, focusing on its main points without losing the impact intended by the speaker. Consider using bullet points for readability.

- Stay faithful to the speaker's intent/perspective.

- When listing things (e.g., a list of examples that the speaker gives) you should use bullet points to make it easier to read.

"""
{transcript}
"""
'''

max_tokens = 4000 - len(tokenizer(prompt)['input_ids'])

summary = openai.Completion.create(
  model="text-davinci-003",
  temperature=0.2,
  prompt=prompt,
  max_tokens=max_tokens
)['choices'][0]['text'].strip()

display(Markdown(summary))

# Next.js 13.1: What's New?

## Create Next App

- The Create Next App has been redesigned and revamped in 13.1, with a fancy new design and the use of newer Next.js features like Next font optimization.
- To get started with Create Next App, run `npx create-next-app` in your terminal.

## App Directory Improvements

- The app directory in 13.1 includes improvements to both the routing and the data fetching system.
- The wrapping element around nested layouts has been removed.
- The amount of client side JavaScript needed inside of the app directory has been reduced by 20 kilobytes.
- The new `modularizeImports` option allows for remapping of imports from named exports to full paths.

## Edge Runtime for API Routes

- The now stable edge runtime is a lighter node.js runtime built on web standard APIs.
- It allows for a strict subset of all of the available node.js APIs that align with the web platform and edge computing platforms.
- Edge functions on Vercel are now generally available.

## Middleware Improvements

- Middleware allows for full flexibility over routing in Next.js.
- It can be used to check if a user has access to a specific resource, modify the request headers for use in get server side props or in an API route, and produce a response.

## Turbo Pack Support

- Support for using Next.js with Turbo Pack, an incremental bundler built with Rust, has been added in 13.1.
- This includes Tailwind support, Post CSS support, Next image support, and stability and reliability improvements.
- To try it out, use the `--turbo` flag in your package.json.